In [3]:
import os
import random

def combine_unique_fens_skip(input_folder, output_file, max_fens=3000):
    fen_set = set()  # To store unique FENs

    # Iterate through all .txt files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            file_path = os.path.join(input_folder, filename)
            with open(file_path, 'r') as file:
                # Skip the first 10 FENs
                all_fens = file.readlines()
                fens_to_process = all_fens[10:]  # Skip the first 10 lines
                
                # Add the remaining FENs to the set (ensures uniqueness)
                for fen in fens_to_process:
                    fen = fen.strip()
                    if fen:  # Ignore empty lines
                        fen_set.add(fen)
    
    # Shuffle and select up to `max_fens` unique FENs
    unique_fens = list(fen_set)
    random.shuffle(unique_fens)
    selected_fens = unique_fens[:max_fens]

    # Write the selected FENs to the output file
    with open(output_file, 'w') as output:
        output.write("\n".join(selected_fens))
    
    print(f"Combined {len(selected_fens)} unique FENs into {output_file}")

# Usage
input_folder = "Game"  # Replace with the folder containing FEN .txt files
output_file = "fens.txt"  # Replace with the desired output file path
combine_unique_fens_skip(input_folder, output_file)


Combined 3000 unique FENs into fens.txt


In [5]:
from PIL import Image, ImageDraw
import chess
import os
import random

def overlay_pieces(board_image, fen, piece_set_path, output_path):
    board = chess.Board(fen)
    piece_images = {piece[0]: Image.open(os.path.join(piece_set_path, piece)) 
                    for piece in os.listdir(piece_set_path) 
                    if piece.endswith(".png")}
    
    # Board square dimensions
    square_size = board_image.width // 8
    
    draw = ImageDraw.Draw(board_image)
    for square, piece in board.piece_map().items():
        row, col = divmod(square, 8)
        piece_img = piece_images[str(piece)]
        piece_img = piece_img.resize((square_size, square_size), Image.ANTIALIAS)
        board_image.paste(piece_img, (col * square_size, (7 - row) * square_size), piece_img)
    
    board_image.save(output_path)

def generate_boards_with_overlay(folder_path, output_board_dir, piece_sets_dir):
    os.makedirs(output_board_dir, exist_ok=True)
    count = 0
    piece_set_folders = [os.path.join(piece_sets_dir, f) for f in os.listdir(piece_sets_dir)]

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            fen_file_path = os.path.join(folder_path, filename)

            with open(fen_file_path, 'r') as file:
                fens = [line.strip() for line in file if line.strip()]

                for fen in fens:
                    count += 1
                    try:
                        # Create a plain board image
                        board_image = Image.new("RGB", (416, 416), "white")
                        draw = ImageDraw.Draw(board_image)

                        # Draw the board grid
                        square_size = 416 // 8
                        for row in range(8):
                            for col in range(8):
                                fill_color = "#F0D9B5" if (row + col) % 2 == 0 else "#B58863"
                                draw.rectangle([col * square_size, row * square_size,
                                                (col + 1) * square_size, (row + 1) * square_size],
                                               fill=fill_color)
                        
                        # Randomly select a piece set
                        selected_piece_set = random.choice(piece_set_folders)
                        output_path = os.path.join(output_board_dir, f"board_{count}.png")
                        
                        overlay_pieces(board_image, fen, selected_piece_set, output_path)
                        
                        print(f"Saved board image: {output_path}")
                    except Exception as e:
                        print(f"Error processing FEN at line {count}: {e}")

folder_path = "singelfen"
output_board_directory = "output"
piece_sets_directory = "image"

generate_boards_with_overlay(folder_path, output_board_directory, piece_sets_directory)


Error processing FEN at line 1: [WinError 267] The directory name is invalid: 'image\\br.png'


In [10]:
from PIL import Image, ImageDraw
import chess
import os

# Mapping standard chess piece notation to custom filenames
PIECE_FILENAMES = {
    "P": "wp", "N": "wn", "B": "wb", "R": "wr", "Q": "wq", "K": "wk",  # White pieces
    "p": "bp", "n": "bn", "b": "bb", "r": "br", "q": "bq", "k": "bk"   # Black pieces
}

def overlay_custom_pieces(board_image, fen, piece_set_path, output_path):
    board = chess.Board(fen)
    piece_images = {key: Image.open(os.path.join(piece_set_path, f"{value}.png"))
                    for key, value in PIECE_FILENAMES.items()}
    
    # Board square dimensions
    square_size = board_image.width // 8

    for square, piece in board.piece_map().items():
        row, col = divmod(square, 8)
        piece_img = piece_images[piece.symbol()]
        piece_img = piece_img.resize((square_size, square_size), Image.Resampling.LANCZOS)
        board_image.paste(piece_img, (col * square_size, (7 - row) * square_size), piece_img)

    board_image.save(output_path)

def generate_boards_with_custom_filenames(folder_path, output_board_dir, piece_set_path):
    os.makedirs(output_board_dir, exist_ok=True)
    count = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            fen_file_path = os.path.join(folder_path, filename)

            with open(fen_file_path, 'r') as file:
                fens = [line.strip() for line in file if line.strip()]

                for fen in fens:
                    count += 1
                    try:
                        # Create a plain board image
                        board_image = Image.new("RGB", (416, 416), "white")
                        draw = ImageDraw.Draw(board_image)

                        # Draw the board grid
                        square_size = 416 // 8
                        for row in range(8):
                            for col in range(8):
                                fill_color = "#F0D9B5" if (row + col) % 2 == 0 else "#B58863"
                                draw.rectangle([col * square_size, row * square_size,
                                                (col + 1) * square_size, (row + 1) * square_size],
                                               fill=fill_color)
                        
                        output_path = os.path.join(output_board_dir, f"board_{count}.png")
                        
                        overlay_custom_pieces(board_image, fen, piece_set_path, output_path)
                        
                        print(f"Saved board image: {output_path}")
                    except Exception as e:
                        print(f"Error processing FEN at line {count}: {e}")

folder_path = "singelfen"
output_board_directory = "output"
piece_set_directory = "image"

generate_boards_with_custom_filenames(folder_path, output_board_directory, piece_set_directory)


Saved board image: output\board_1.png


In [11]:
from PIL import Image
import chess
import os

# Mapping standard chess piece notation to custom filenames
PIECE_FILENAMES = {
    "P": "wp", "N": "wn", "B": "wb", "R": "wr", "Q": "wq", "K": "wk",  # White pieces
    "p": "bp", "n": "bn", "b": "bb", "r": "br", "q": "bq", "k": "bk"   # Black pieces
}

def overlay_custom_pieces(board_image, fen, piece_set_path, output_path):
    board = chess.Board(fen)
    piece_images = {key: Image.open(os.path.join(piece_set_path, f"{value}.png"))
                    for key, value in PIECE_FILENAMES.items()}
    
    # Board square dimensions
    square_size = board_image.width // 8

    for square, piece in board.piece_map().items():
        row, col = divmod(square, 8)
        piece_img = piece_images[piece.symbol()]
        piece_img = piece_img.resize((square_size, square_size), Image.Resampling.LANCZOS)
        board_image.paste(piece_img, (col * square_size, (7 - row) * square_size), piece_img)

    board_image.save(output_path)

def generate_boards_with_custom_background(folder_path, output_board_dir, piece_set_path, board_image_path):
    os.makedirs(output_board_dir, exist_ok=True)
    count = 0

    # Load the custom board background
    base_board_image = Image.open(board_image_path)
    base_board_image = base_board_image.resize((416, 416), Image.Resampling.LANCZOS)

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            fen_file_path = os.path.join(folder_path, filename)

            with open(fen_file_path, 'r') as file:
                fens = [line.strip() for line in file if line.strip()]

                for fen in fens:
                    count += 1
                    try:
                        # Create a copy of the base board image for each FEN
                        board_image = base_board_image.copy()
                        
                        output_path = os.path.join(output_board_dir, f"board_{count}.png")
                        
                        overlay_custom_pieces(board_image, fen, piece_set_path, output_path)
                        
                        print(f"Saved board image: {output_path}")
                    except Exception as e:
                        print(f"Error processing FEN at line {count}: {e}")

folder_path = "singelfen"
output_board_directory = "output"
piece_set_directory = "image"
board_image_path = "board.png"  # Path to your custom board image

generate_boards_with_custom_background(folder_path, output_board_directory, piece_set_directory, board_image_path)


Saved board image: output\board_1.png
